## Обучение опций 
Нашей задачей будет создание набора опций, каждая из которых должна быть обучена достигать определенные состояния в задаче такси. Для обучения мы будем использовать QLearningAgent, которого мы написали на одном из прошлых семинаров. 

In [158]:
# импортируем файлы и создаем окружение
import gym
import random
from collections import defaultdict

import numpy as np

environment = gym.make('Taxi-v2')
environment.render()


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [159]:
# создаем классс для Q-агента
class QLearningAgent:
    def __init__(self, alpha, epsilon, gamma, get_legal_actions):
        self.get_legal_actions = get_legal_actions
        self._q_values = defaultdict(lambda: defaultdict(lambda: 0))  # when called, non-existent values appear as zeros
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma

    def get_q_value(self, state, action):
        """
          Returns Q(state,action)
        """
        return self._q_values[state][action]

    def set_q_value(self, state, action, value):
        """
          Sets the Qvalue for [state,action] to the given value
        """
        self._q_values[state][action] = value

    def get_value(self, state):
        """
          Returns max_action Q(state,action)
          where the max is over legal actions.
        """

        possible_actions = self.get_legal_actions(state)
        # If there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0

        value = max([self.get_q_value(state, action) for action in possible_actions])
        return value

    def get_policy(self, state):
        """
          Compute the best action to take in a state.

        """
        possible_actions = self.get_legal_actions(state)

        # If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        best_action = None

        for action in possible_actions:
            if best_action is None:
                best_action = action
            elif self.get_q_value(state, action) > self.get_q_value(state, best_action):
                best_action = action

        return best_action

    def get_action(self, state):
        """
          Compute the action to take in the current state, including exploration.

          With probability self.epsilon, we should take a random action.
          otherwise - the best policy action (self.getPolicy).

        """

        #
        possible_actions = self.get_legal_actions(state)

        # если в текущей ситуации нет возможных действий - возвращаем None
        if len(possible_actions) == 0:
            return None

        if np.random.random() < self.epsilon:
            action = random.choice(possible_actions)
        else:
            action = self.get_policy(state)
        return action

    def update(self, state, action, next_state, reward):
        t = self.alpha * (reward + self.gamma * self.get_value(next_state) - self.get_q_value(state, action))
        reference_qvalue = self.get_q_value(state, action) + t
        self.set_q_value(state, action, reference_qvalue)


### Задание 1 
Разберемся как реализована среда Taxi: https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

Создадим 4 окружения аналогичных Taxi, в которых целью агента будет достижение одной из точек: R, G, B, Y соответственно. 

In [160]:
class TaxiStepWrapper(gym.Wrapper):
    def __init__(self, env, target_id, target_reward):
        super().__init__(env)
        self._target = target_id
        self._target_reward = target_reward

    def _step(self, action):

        # получаем изначальные параметры (state, reward, _, obs), которые передает среда, используя метод step 
        # проверяем является ли полученнуе состояние завершающим для нашего модифицированного окружения
        # изменяем вознаграждение (reward) и флаг завершения эпизода (is_done)
        # за каждое действие будем давать вознаграждение -1, за достижение цели - self._target_reward
        # Ваш код здесь

        return state, reward, is_done, obs


Проверим нашу обертку (wrapper), используя случайную стратегию.  Порядок точек должен быть  R, G, Y, B.

In [161]:
for target in range(4):


    # создаем окружение с заданным целевым состоянием
    # Ваш код здесь

    
    # применяем случайную стратегию, пока эпизод не завершится
    # Ваш код здесь


    wrapped_env.render()
    print("state:{s} reward:{r}\n".format(**locals()))


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
state:4 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
state:94 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
state:411 reward:50

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
state:471 reward:50



In [162]:
# воспользуемся методом play_and_train, который мы реализовали на прошлом семинаре
def play_and_train(env, agent, t_max=10 ** 4):
    total_discounted_reward = 0.0
    s = env.reset()
    for t in range(t_max):
        a = agent.get_action(s)
        next_s, r, done, _ = env.step(a)
        agent.update(s, a, next_s, r)
        s = next_s
        total_discounted_reward += r
        if done:
            break
    return total_discounted_reward


### Задание 2 
1. Обучим агентов на созданных нами окружениях.
2. Создадим упрощенный вариант опций, каждая опция будет иметь стратегию, множество начальных состояний и множество конечных состояний.

In [163]:
n_actions = environment.action_space.n

# параметры, которые будут использовать агенты
params = {"alpha": 0.1, "epsilon": 0.1, "gamma": 0.99, "get_legal_actions": lambda s: range(4)}

# создаем агентов 
agents_for_options = [QLearningAgent(**params) for _ in range(4)]

for index in range(4):


    # создаем окружение с заданным целевым состоянием
    # Ваш код здесь



    # используя созданных окружения обучаем на них агентов
    # Ваш код здесь

In [164]:
# реализуем класс опции
class Option:
    def __init__(self, policy, termination_prob, initial):
        self.policy = policy
        self.termination_prob = termination_prob
        self.initial_states = initial

    def can_start(self, state):
        return state in self.initial_states

    def terminate(self, state):
        return random.random() <= self.termination_prob[state]

    def get_action(self, state):
        return self.policy.get_action(state)


In [165]:
options = []
for index, agent in enumerate(agents_for_options):


    # Создаем словарь termination_prob, в котором каждому состоянию окружения нужно задать вероятность завершения опции
    # В нашем случае зададим 1.0 или 0.0, в зависимости от состояния
    # Ваш код здесь



    # Создаем множество (set) initial, в которое добавляем все состояния, из которых опция может быть вызвана (все кроме целевых)
    # Ваш код здесь

    options.append(Option(policy=agent, termination_prob=termination_prob, initial=initial))


### Задание 3
Напишем функцию, которая будет запускать опцию и возвращать дисконтированное вознаграждение, опираясь на число совершенных действий
$$ R = r_{1} + \gamma r_{2} + \gamma^{2} r_{3} + \dots + \gamma^{t-1}r_{t}$$

In [166]:
def apply_option(s, option, gamma, env, debug=False):
    reward = 0
    steps = 0


    # Если опция не может быть запущена - возвращаем None
    # Ваш код здесь




    # Взаимодействуем со средой пока опция (используем методм terminate) или окружение не завершится
    # Считаем дисконтированное вознаграждение reward (используем steps)
    # Также добавим render окружения, если включен флаг - debug.
    # Ваш код здесь


    return reward


In [167]:
# проверим работу метода
env = gym.make('Taxi-v2')
s = env.reset()

r = apply_option(s, options[0], 0.99, env, debug=True)


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Option terminated
SMDP reward: -1.0


Кажется, что все хорошо, но мы забыли рассмотреть вариант, когда пассажир может находиться в такси! Переведем среду в состояние, где пассажира мы уже подобрали и посмотрим, как ведет себя  одна из опций.

In [1]:
s = env.reset()
env.unwrapped.s = 499
env.render()
print("\n" * 2)
r = apply_option(s, options[0], 0.99, env, debug=True)


NameError: name 'env' is not defined

### Задание 4
Видим, что опции не обучились действовать в такой ситуации. 
Исправим нашу функцию обучения так, чтобы опции работали корректно для всех возможных состояний среды и сгенерируем их заново.

In [169]:
def play_and_train_modified(env, agent, t_max=10 ** 4):
    # Зададим новую функцию play_and_train, которая в качестве начального состояния выбирает любое состояние среды,
    # включая и то, когда пассажир уже находится в такси

    total_discounted_reward = 0.0
    s = env.reset()



    # Выбираем случайное состояние среды, включае то, где пассажир уже в машине (используем метод env.uwrapped)
    # Ваш код здесь

    for t in range(t_max):
        a = agent.get_action(s)
        next_s, r, done, _ = env.step(a)
        agent.update(s, a, next_s, r)
        s = next_s
        total_discounted_reward += r
        if done:
            break
    return total_discounted_reward


for index in range(4):
    for _ in range(5250):
        wrapped_env = TaxiStepWrapper(env=environment, target_id=index, target_reward=50)
        play_and_train_modified(env=wrapped_env, agent=agents_for_options[index])


Запустим данную ячейку несколько раз и убедимся, что агент обучился для всех случаев!

In [185]:
env = environment
s = env.reset()

env.unwrapped.s = random.randint(0, 499)
apply_option(s, options[0], 0.99, env, debug=True)


+---------+
|R: | : :G|
| : : : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Option terminated
SMDP reward: -3.940399


-3.940399

### Бонус 
Реализуйте иерархию, используя элементарные (опции из одного действия) и обученные опции.